In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
%matplotlib inline

In [164]:
df = pd.read_csv('train.csv',delimiter=',',encoding='latin-1')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [171]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saima\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [177]:
import re
def clean_punctuations(marks):
    return re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", marks)


def remove_stopwords(text):
   text = " ".join([word for word in text.split() if word not in (stop)])
   return text


def convert_to_lowercase(string):
    return string.lower()


In [178]:
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(convert_to_lowercase)
df['text'] = df['text'].apply(clean_punctuations)


In [179]:
df['text']

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [146]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [147]:
count_vectorizer = feature_extraction.text.CountVectorizer()


In [149]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors -
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])


In [ ]:
print(count_vectorizer.get_feature_names())


In [134]:

count_vectorizer.vocabulary_


{'our': 14003,
 'deeds': 5490,
 'are': 2192,
 'the': 18669,
 'reason': 15678,
 'of': 13681,
 'this': 18777,
 'earthquake': 6379,
 'may': 12141,
 'allah': 1852,
 'forgive': 7661,
 'us': 19774,
 'all': 1851,
 'forest': 7652,
 'fire': 7439,
 'near': 13122,
 'la': 11091,
 'ronge': 16266,
 'sask': 16611,
 'canada': 3843,
 'residents': 15940,
 'asked': 2312,
 'to': 18971,
 'shelter': 17022,
 'in': 9718,
 'place': 14612,
 'being': 2900,
 'notified': 13423,
 'by': 3698,
 'officers': 13701,
 'no': 13335,
 'other': 13987,
 'evacuation': 6909,
 'or': 13919,
 'orders': 13936,
 'expected': 7014,
 '13': 176,
 '000': 1,
 'people': 14389,
 'receive': 15699,
 'wildfires': 20607,
 'california': 3797,
 'just': 10550,
 'got': 8364,
 'sent': 16870,
 'photo': 14500,
 'from': 7823,
 'ruby': 16379,
 'alaska': 1800,
 'as': 2280,
 'smoke': 17394,
 'pours': 14821,
 'into': 9947,
 'school': 16700,
 'rockyfire': 16219,
 'update': 19722,
 'hwy': 9399,
 '20': 343,
 'closed': 4462,
 'both': 3346,
 'directions': 5819,

In [138]:
count_vectorizer._count_vocab

<bound method CountVectorizer._count_vocab of CountVectorizer()>

In [180]:
df.drop(['keyword', 'location', 'id'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7613 non-null   object
 1   target  7613 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.1+ KB


In [181]:
X = df.text
Y = df.target
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [182]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=42)

In [191]:
from keras.utils import pad_sequences
max_words = 1500
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [192]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model


In [193]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_8 (Embedding)     (None, 150, 50)           75000     
                                                                 
 lstm_8 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_16 (Activation)  (None, 256)               0         
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [194]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.20,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
41/41 [==============================] - 11s 189ms/step - loss: 0.6217 - accuracy: 0.6582 - val_loss: 0.5157 - val_accuracy: 0.7730
Epoch 2/10
41/41 [==============================] - 6s 154ms/step - loss: 0.4213 - accuracy: 0.8170 - val_loss: 0.4731 - val_accuracy: 0.7969


In [187]:
# test_df = pd.read_csv("./test.csv")
# X_test = test_df.text
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [188]:
result = model.predict(test_sequences_matrix)

36/36 [==============================] - 2s 23ms/step


In [189]:
result=np.rint(result)
accr = model.evaluate(result,Y_test)

36/36 [==============================] - 1s 3ms/step - loss: 0.7139 - accuracy: 0.5701


In [190]:
len(result)

1142

In [195]:
sample_submission = pd.read_csv("./sample_submission.csv")
sample_submission["target"] = result

ValueError: Length of values (1142) does not match length of index (3263)

In [94]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.656
  Accuracy: 0.792
